<a href="https://colab.research.google.com/github/DeepSmeag/ML_Colab/blob/main/AND%2BXORperceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>